In [7]:
from __future__ import print_function
import pandas as pd
import matplotlib.pyplot as plt

from lime.lime_tabular import RecurrentTabularExplainer
from tqdm import tqdm
import keras
from sp_modif.model_function import *
from sp_modif.methods import *
from sp_modif.data_prep import *
from sp_modif.evaluator import *
from sp_modif.SHAP import *
from sp_modif.L2X import *
import warnings

warnings.filterwarnings('ignore')
print("okay")

okay


In [8]:
SEED = 0
def set_seed(seed=SEED):
    os.environ['PYTHONHASHSEED'] = str(SEED)
    random.seed(SEED)
    np.random.seed(SEED)
    tf.random.set_seed(SEED)

# Appeler la fonction pour fixer le seed
set_seed(SEED)

In [9]:
# Function to generate explanations
# Function explanation for lime

def get_lime_explanation(data, e, L2X=False) :
    # e  = fn = model.predict
    # Iniatialisation
    df_expplanation = pd.DataFrame(columns=[str(i) for i in range(data.shape[1]*data.shape[2])])

    # Get explanations
    for row in range(data.shape[0]) : 
        explanation = lime_explainer.explain_instance(data[row],
                                                      e,
                                                      num_features=data.shape[1]*data.shape[2]) 
        # fn = model.predict, initialize lime_explainer = Reccurent()
        lime_values = explanation.local_exp[1]
        # Add explanation in df_explanation
        lime_dict = {}
        for tup in lime_values :
            lime_dict[str(tup[0])] = tup[1]
        df_expplanation.loc[len(df_expplanation)] = lime_dict
    
    return df_expplanation

# # Function explanation for others
def get_explainations(data, e, L2X = False) :
    
    # df diemnsion
    if L2X==True :
        X_to_def_col = data[0:1]
        explanation_test = e.explain(X_to_def_col.reshape((X_to_def_col.shape[0], -1)))
        num_columns = explanation_test.flatten().shape[0]
        
    else : 
        explanation_test = e.explain(data[0:1])
        num_columns = explanation_test.flatten().shape[0]
    
    # Iniatialisation
    df_expplanation = pd.DataFrame(columns=[str(i) for i in range(num_columns)])

    # Get explanations
    for row in range(data.shape[0]) :
        if L2X==True:
            X_row = data[row:row+1]
            explanation = e.explain(X_row.reshape((X_row.shape[0], -1)))
        else :
            explanation = e.explain(data[row:row+1])
        # Add explanation in df_explanation
        explanation = explanation.flatten()
        feature_dict = {}
        for i in range(num_columns) :
            feature_dict[str(i)] = explanation[i]
        df_expplanation.loc[len(df_expplanation)] = feature_dict
    
    return df_expplanation

In [10]:
# Data preparation
# Data loading
train, test, y_test = prepare_data('FD003.txt')
print(train.shape, test.shape, y_test.shape)
# train['RUL'].clip(upper=125, inplace=True)
sensor_names = ['T20','T24','T30','T50','P20','P15','P30','Nf','Nc','epr','Ps30','phi',
                'NRf','NRc','BPR','farB','htBleed','Nf_dmd','PCNfR_dmd','W31','W32']

remaining_sensors = ['T24','T30','T50','P30','Nf','Nc','Ps30','phi',
                'NRf','NRc','BPR','htBleed','W31','W32'] # selection based on main_notebook

drop_sensors = [element for element in sensor_names if element not in remaining_sensors] 

(24720, 27) (16596, 26) (100, 1)


In [11]:
def model003(input_shape,weights_file, nb_layers = None):
    nodes_per_layer = 32
    activation_value= 'tanh'
    dropout = 0.4
    bs = 64
    if nb_layers==1:
        '''
        node = 256, activation = tanh, dropout = 0.3, bs = 64
        
        '''
        '''
        nodes_per_layer = 256
        activation_value= 'tanh'
        dropout = 0.4
        bs = 64
        '''

        model = Sequential()
        model.add(LSTM(nodes_per_layer, activation=activation_value, input_shape=input_shape))
        model.add(Dropout(dropout))
        model.add(Dense(1))
        
    elif nb_layers == 2 :
        model = Sequential()
        model.add(LSTM(32, activation='tanh', input_shape=input_shape, return_sequences=True))
        model.add(LSTM(64, activation='tanh'))
        model.add(Dropout(dropout))
        model.add(Dense(256, activation = 'relu'))
        model.add(Dropout(dropout))
        model.add(Dense(1))
        
    elif nb_layers==3:
        model = Sequential()
        model.add(LSTM(32, activation='tanh', input_shape=input_shape, return_sequences=True))
        model.add(LSTM(64, activation='tanh', return_sequences=True))
        Dropout(dropout)
        # Dense(256, activation = 'relu')
        model.add(LSTM(32, activation='tanh'))
        model.add(Dense(256, activation = 'relu'))
        model.add(Dropout(dropout))
        model.add(Dense(1))
    
    elif nb_layers==4:
        model = Sequential()
        model.add(LSTM(32, activation='tanh', input_shape=input_shape, return_sequences=True))
        model.add(LSTM(64, activation='tanh', return_sequences=True))
        model.add(Dropout(0.2))
        model.add(LSTM(64, activation='tanh', return_sequences=True))
        model.add(LSTM(32, activation='tanh'))
        model.add(Dense(256, activation = 'relu'))
        model.add(Dropout(dropout))
        model.add(Dense(1))
        
    model.compile(loss='mean_squared_error', optimizer=Adam(learning_rate=0.001))
    model.save_weights(weights_file)

    return model

In [13]:
# Data preparation
# Data prepration
sequence_length = 30
alpha = 0.3
upper = 125
    
learning_rate_ = 0.001
dropout = 0.4
activation = 'tanh'
epochs = 20
batch_size = 64

results = pd.DataFrame()
df_metrics = pd.DataFrame()
    
train = rul_piecewise_fct(train, upper)

X_train_interim, X_test_interim = prep_data(train, test, drop_sensors, remaining_sensors, alpha)
# create sequences train, test
train_array = gen_data_wrapper(X_train_interim, sequence_length,remaining_sensors)
label_array = gen_label_wrapper(X_train_interim, sequence_length, ['RUL'])

test_gen = (list(gen_test_data(X_test_interim[X_test_interim['Unit']==unit_nr], sequence_length,remaining_sensors, -99.))
               for unit_nr in X_test_interim['Unit'].unique())
    
test_array = np.concatenate(list(test_gen)).astype(np.float32)
test_rul = rul_piecewise_fct(y_test,upper)
print(train_array.shape, label_array.shape, test_array.shape)



for md in tqdm.tqdm(range(2,4)):
    #Model creation
    weights_file = "fd003_result/" + str(md) + 'lstm_hyper_parameter_weights.h5'
    input_shape = (sequence_length, len(remaining_sensors))
    model = model003(input_shape=input_shape, weights_file=weights_file, nb_layers=md)
        
    mse_val = []
    R2_val = []
    RMSE = []
    score_val = []
    mse = []
    # Model fitting
    with tf.device('/device:GPU:0'):
        history = model.fit(train_array, label_array,
                                validation_data=(test_array, test_rul),
                                epochs=epochs,
                                batch_size=batch_size,
                                # callbacks=[cb],
                                verbose=1)
        mse.append(history.history['val_loss'][-1])

        y_hat_val_split = model.predict(test_array)
        R2_val.append(r2_score(test_rul, y_hat_val_split))
        RMSE.append(np.sqrt(mean_squared_error(test_rul, y_hat_val_split)))
        score_val.append(compute_s_score(test_rul, y_hat_val_split))

        #  append results
    d = {'RMSE' :np.mean(RMSE), 'std_RMSE' :np.std(RMSE),
         'S_score' :np.mean(score_val), 'std_S_score' :np.std(score_val),
         'MSE':np.mean(mse), 'std_MSE':np.std(mse),
        #  'nodes':str(nodes_per_layer),
         'dropout':dropout, 'activation':activation, 'batch_size':batch_size,
         'TW' : sequence_length, 'alpha' : alpha, 'upper' : upper}
    results = pd.concat([results, pd.DataFrame(d, index=[0])], ignore_index=True)
    results.to_csv('fd003_result/nblayers/performance23.csv')
    
    with tf.device('/device:GPU:0'):
            for rd in range(5):

                # Get explanation and calculate the score
                # Echantillonage
                n_individus = test_array.shape[0]

                # # Choisir aléatoirement 5 indices d'individus
                np.random.seed(rd)
                indices_choisis = np.random.choice(n_individus, size=5, replace=False)

                # Sélectionner les données correspondant aux indices choisis
                test_array_sampling = test_array[indices_choisis, :, :]
                label_array_sampling = y_test.values[indices_choisis, :]

                # Afficher les dimensions des données sélectionnées
                print(test_array_sampling.shape, label_array_sampling.shape)
                
                # distance matrix XX'
                X_dist = pd.DataFrame(squareform(pdist(test_array_sampling.reshape((test_array_sampling.shape[0], -1)))))

                # LIME
                lime_explainer = RecurrentTabularExplainer(test_array, training_labels=label_array,
                                                                feature_names=remaining_sensors,
                                                                mode = 'regression',
                                                                )
                lime_values = get_lime_explanation(test_array_sampling, e = model.predict)
                lime_values.shape
                Lime_dist = pd.DataFrame(squareform(pdist(lime_values))) # Lime values explanation matrix

                #Lime's metrics
                list_metrics_lime = {}
                list_metrics_lime['identity'] = identity(X_dist, Lime_dist)
                list_metrics_lime['separability'] = separability(X_dist, Lime_dist)
                list_metrics_lime['stability'] = stability(X_dist, Lime_dist)
                list_metrics_lime['coherence'], list_metrics_lime['completness'], list_metrics_lime['congruence'] = coherence(model=model.predict, 
                                                                explainer = get_lime_explanation,
                                                            samples=test_array_sampling,
                                                                targets=label_array, e = model.predict)
                list_metrics_lime['selectivity'] = selectivity(model=model.predict, explainer = get_lime_explanation,
                                                            samples=test_array_sampling, e_x = model.predict)
                list_metrics_lime['accumen'] = acumen(get_lime_explanation, test_array_sampling, e=model.predict)
                list_metrics_lime['Verm_stability'] = stability_Velmurugan(get_lime_explanation, test_array_sampling,
                                                                            e=model.predict, top_features=200)
                list_metrics_lime['fidelity'], list_metrics_lime['sparsity'] = fidelity(model=model.predict, 
                                                                explainer = get_lime_explanation,
                                                                samples=test_array_sampling,
                                                                e = model.predict, L2X=True)
                list_metrics_lime['instability'] = instability(model=model.predict, 
                                                                explainer = get_lime_explanation,
                                                                samples=test_array_sampling,
                                                                e = model.predict, L2X=True)
                
                list_metrics_lime['alpha'] = alpha
                list_metrics_lime['upper'] = upper
                list_metrics_lime['sequence_length'] = sequence_length
                list_metrics_lime['explainer'] = 'lime'
                list_metrics_lime['model'] = md

                df_metrics = pd.concat([df_metrics, pd.DataFrame([list_metrics_lime])])

                # SHAP
                e = KernelSHAP(model)
                shapvalues = get_explainations(test_array_sampling, e)
                shapvalues.shape

                list_metrics_shap = {}
                shap_dist = pd.DataFrame(squareform(pdist(shapvalues))) # shap values explanation matrix

                list_metrics_shap['identity'] = identity(X_dist, shap_dist)
                list_metrics_shap['separability'] = separability(X_dist, shap_dist)
                list_metrics_shap['stability'] = stability(X_dist, shap_dist)
                list_metrics_shap['coherence'], list_metrics_shap['completness'], list_metrics_shap['congruence'] = coherence(model=model.predict, 
                                                                explainer = get_explainations,
                                                            samples=test_array_sampling,
                                                                targets=label_array, e = e)
                list_metrics_shap['selectivity'] = selectivity(model=model.predict, explainer = get_explainations,
                                                samples=test_array_sampling, e_x=e)
                list_metrics_shap['accumen'] = acumen(get_explainations, test_array_sampling, e=e)
                list_metrics_shap['Verm_stability'] = stability_Velmurugan(get_explainations, test_array_sampling,
                                                                            e=e, top_features=200)
                list_metrics_shap['fidelity'], list_metrics_shap['sparsity']= fidelity(model=model.predict, 
                                                                explainer = get_explainations,
                                                                samples=test_array_sampling,
                                                                e = e)
                list_metrics_shap['instability']= instability(model=model.predict, 
                                                                explainer = get_explainations,
                                                                samples=test_array_sampling,
                                                                e = e)
                
                list_metrics_shap['alpha'] = alpha
                list_metrics_shap['upper'] = upper
                list_metrics_shap['sequence_length'] = sequence_length
                list_metrics_shap['explainer'] = 'shap'
                list_metrics_shap['model'] = md

                df_metrics = pd.concat([df_metrics, pd.DataFrame([list_metrics_shap])])
                
                # L2X
                e = L2X(model.predict, test_array_sampling)
                l2xvalues = get_explainations(test_array_sampling, e, L2X=True)
                l2xvalues.shape

                # L2X's metrics
                list_metrics_l2x = {}
                l2x_dist = pd.DataFrame(squareform(pdist(l2xvalues))) # Lime values explanation matrix

                list_metrics_l2x['identity'] = identity(X_dist, l2x_dist)
                list_metrics_l2x['separability'] = separability(X_dist, l2x_dist)
                list_metrics_l2x['stability'] = stability(X_dist, l2x_dist)
                list_metrics_l2x['coherence'], list_metrics_l2x['completness'], list_metrics_l2x['congruence'] = coherence(model=model.predict, explainer = get_explainations,
                                                            samples=test_array_sampling, targets=label_array_sampling, e = e, L2X=True)
                list_metrics_l2x['selectivity'] = selectivity(model=model.predict, explainer = get_explainations,
                                                samples=test_array_sampling, e_x=e, L2X=True)
                list_metrics_l2x['accumen'] = acumen(get_explainations, test_array_sampling, e=e, L2X=True)
                list_metrics_l2x['Verm_stability'] = stability_Velmurugan(get_explainations, test_array_sampling,
                                                                            e=e, top_features=200, L2X=True)
                list_metrics_l2x['fidelity'], list_metrics_l2x['sparsity']= fidelity(model=model.predict, 
                                                                explainer = get_explainations,
                                                                samples=test_array_sampling,
                                                                e = e, L2X=True)
                list_metrics_l2x['instability'] = instability(model=model.predict, 
                                                                explainer = get_explainations,
                                                                samples=test_array_sampling,
                                                                e = e, L2X=True)
                
                list_metrics_l2x['alpha'] = alpha
                list_metrics_l2x['upper'] = upper
                list_metrics_l2x['sequence_length'] = sequence_length
                list_metrics_l2x['explainer'] = 'l2x'
                list_metrics_l2x['model'] = md

                df_metrics = pd.concat([df_metrics, pd.DataFrame([list_metrics_l2x])])
            df_metrics.to_csv('fd003_result/nblayers/quality_by_layer23.csv')

(21820, 30, 14) (21820, 1) (100, 30, 14)


  0%|          | 0/2 [00:00<?, ?it/s]


Epoch 1/20

341/341 [==============================] - 17s 32ms/step - loss: 2566.6951 - val_loss: 1690.7837
Epoch 2/20
341/341 [==============================] - 9s 28ms/step - loss: 816.4733 - val_loss: 397.8224
Epoch 3/20
341/341 [==============================] - 10s 29ms/step - loss: 450.1246 - val_loss: 340.5989
Epoch 4/20
341/341 [==============================] - 11s 31ms/step - loss: 417.3625 - val_loss: 363.7005
Epoch 5/20
341/341 [==============================] - 10s 28ms/step - loss: 386.8178 - val_loss: 313.3810
Epoch 6/20
341/341 [==============================] - 9s 27ms/step - loss: 373.8385 - val_loss: 282.7762
Epoch 7/20
341/341 [==============================] - 10s 31ms/step - loss: 342.2943 - val_loss: 299.7641
Epoch 8/20
341/341 [==============================] - 10s 30ms/step - loss: 337.7661 - val_loss: 270.4546
Epoch 9/20
341/341 [==============================] - 12s 35ms/step - loss: 326.3235 - val_loss: 250.5268
Epoch 10/20
341/341 [=======================

 50%|█████     | 1/2 [2:59:08<2:59:08, 10748.58s/it]

Epoch 1/20
341/341 [==============================] - 9s 18ms/step - loss: 3050.5596 - val_loss: 1746.6594
Epoch 2/20
341/341 [==============================] - 6s 16ms/step - loss: 1775.0494 - val_loss: 1733.3512
Epoch 3/20
341/341 [==============================] - 6s 16ms/step - loss: 1765.9910 - val_loss: 1738.1938
Epoch 4/20
341/341 [==============================] - 5s 16ms/step - loss: 1772.5593 - val_loss: 1771.8860
Epoch 5/20
341/341 [==============================] - 6s 16ms/step - loss: 680.0008 - val_loss: 286.5606
Epoch 6/20
341/341 [==============================] - 6s 16ms/step - loss: 296.4762 - val_loss: 313.8592
Epoch 7/20
341/341 [==============================] - 5s 16ms/step - loss: 268.2788 - val_loss: 292.5872
Epoch 8/20
341/341 [==============================] - 6s 16ms/step - loss: 258.4162 - val_loss: 241.8099
Epoch 9/20
341/341 [==============================] - 6s 16ms/step - loss: 250.7650 - val_loss: 229.8363
Epoch 10/20
341/341 [==========================

100%|██████████| 2/2 [5:10:11<00:00, 9305.74s/it]   


In [15]:
df_metrics

,identity,separability,stability,coherence,completness,congruence,selectivity,accumen,Verm_stability,fidelity,sparsity,instability,alpha,upper,sequence_length,explainer,model
0,1.0,1.0,1.0,0.299799,0.185532,0.252747,0.708523,0.014797,0.754146,-0.136834,0.023810,0.732466,0.3,125,30,lime,2
0,1.0,1.0,1.0,0.304840,0.069472,0.253071,0.802935,0.384630,1.000000,0.217270,0.023810,0.575758,0.3,125,30,shap,2
0,1.0,1.0,1.0,0.732807,0.056653,0.531142,0.703460,0.023866,1.000000,0.191302,0.023810,0.381818,0.3,125,30,l2x,2
0,1.0,1.0,1.0,0.535610,0.064610,0.406887,0.713396,0.011933,0.707024,-1.899376,0.023810,0.379072,0.3,125,30,lime,2
0,1.0,1.0,1.0,0.541587,0.020818,0.407806,0.811592,0.394081,1.000000,-0.564770,0.023810,0.339661,0.3,125,30,shap,2
0,1.0,1.0,1.0,2.637028,0.095934,1.345106,0.689415,0.049451,1.000000,-0.835029,0.023810,0.371429,0.3,125,30,l2x,2
0,1.0,1.0,1.0,0.085183,0.270681,0.054516,0.760023,0.034654,0.573854,0.267323,0.023810,0.572133,0.3,125,30,lime,2
0,1.0,1.0,1.0,0.089754,0.114357,0.055224,0.797143,0.334415,1.000000,0.405544,0.023810,0.523815,0.3,125,30,shap,2
0,1.0,1.0,1.0,0.150344,0.284743,0.131433,0.790679,0.031408,1.000000,0.391996,0.023810,0.559980,0.3,125,30,l2x,2
0,1.0,1.0,1.0,0.274160,17.441656,0.366776,0.725810,0.007542,0.764390,-1.170543,0.023810,0.589251,0.3,125,30,lime,2
